# Coleta de dados
Nesta seção iremos coletar os dados por meio de web scraping e apis de outros sites

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
prefix = 'https://www.transfermarkt.com'
def getSoup(url):
    res = requests.get(url, headers = headers)
    return BeautifulSoup(res.content, 'html.parser')


### Função para realizar o web scraping das tableas de transferências no transfermarkt

In [6]:
def getClubUrl(url):
    url = url.replace('/transfers', '')  # Remove a substring '/transfers'
    match = re.search(r'/verein/(\d+)', url)
    verein_id = match.group(1)  # Obtém o número após "/verein/"
    new_url = match.string[:match.start()] + \
        '/startseite' '/verein/' + verein_id
    return new_url


def eurValue(string):
    # Expressão regular para verificar se começa com '€' e termina com 'm' ou 'k'
    padrao = r'^€.*[mk]$'
    if re.match(padrao, string):
        return True
    else:
        return False


def eurToInt(string):
    valor_str = re.sub(r'[€mk]', '', string)  # Remove '€', 'm' e 'k' da string
    # Substitui a vírgula pelo ponto como separador decimal e converte para float
    valor_float = float(valor_str.replace(',', '.'))
    if 'm' in string:
        valor_int = int(valor_float * 1000000)  # Converte para milhões
    elif 'k' in string:
        valor_int = int(valor_float * 1000)  # Converte para milhares
    else:
        valor_int = int(valor_float)
    return valor_int


def getTransfersFromUrl(url, year):
    transferTable = []
    page = getSoup(url.format(year))
    tables = page.find_all('div', {'class': 'box'})
    nTeams = int(page.find(lambda tag: tag.name ==
                 "li" and "Number of teams:" in tag.text).find('span').text.split()[0])
    tables = tables[3:nTeams+3]
    for table in tables:
        currTeamUrl = getClubUrl(table.find('a').get('href'))
        currTeam = table.find('a').get('title')
        arrivalsAndDepartures = table.find_all('table')

        arrivals = arrivalsAndDepartures[0].find_all('tr', recursive=True)
        arrivals = arrivals[1:]
        for transfer in arrivals:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubLeftUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubJoinedUrl = currTeamUrl
            clubLeft = transfer[6].find('a').get('title')
            clubJoined = currTeam
            teamTransferType = "out"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

        departures = arrivalsAndDepartures[1].find_all(
            'tr', recursive=True)
        departures = departures[1:]
        for transfer in departures:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubJoinedUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubLeftUrl = currTeamUrl
            clubLeft = currTeam
            clubJoined = transfer[6].find('a').get('title')
            teamTransferType = "in"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

    columns = ['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
               'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl', 'clubJoined', 'clubJoinedUrl', 'teamTransferType']
    final_df = pd.DataFrame(transferTable, columns=columns)
    final_df['season'] = year
    final_df['playerAge'] = final_df['playerAge'].apply(
        lambda x: re.sub(r'\D+', '', str(x)))
    final_df['playerAge'] = final_df['playerAge'].astype(int)
    final_df['valuation'] = final_df['playerMarketValue'] - final_df['transferFee']
    return final_df


### Função para fazer o web scraping das tabelas de transferências de um determinado período e concatenar elas

In [7]:
def getTransfersInRange(begin, end, url):
    curr_df = getTransfersFromUrl(url, begin)
    for i in range(begin+1, end+1):
        curr_df = pd.concat(
            [curr_df, getTransfersFromUrl(url, i)]).reset_index(drop=True)
    return curr_df

### Obtendo dados das transferências para um range de temporadas da Premier League (demora 3 a 6 segundos por temporada, depende da internet)

In [8]:
pl = r'https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
pl_transfers = getTransfersInRange(2000, 2023, pl)
pl_transfers    

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,clubJoinedUrl,teamTransferType,season,valuation
0,Rio Ferdinand,/rio-ferdinand/profil/spieler/3235,22,CB,England,0,26000000,West Ham United,/west-ham-united/startseite/verein/379,Leeds United,/leeds-united/startseite/verein/399,out,2000,-26000000
1,Olivier Dacourt,/olivier-dacourt/profil/spieler/3572,25,DM,France,0,10500000,RC Lens,/rc-lens/startseite/verein/826,Leeds United,/leeds-united/startseite/verein/399,out,2000,-10500000
2,Mark Viduka,/mark-viduka/profil/spieler/3242,24,CF,Australia,0,9150000,Celtic FC,/celtic-glasgow/startseite/verein/371,Leeds United,/leeds-united/startseite/verein/399,out,2000,-9150000
3,Dominic Matteo,/dominic-matteo/profil/spieler/3765,26,LB,Scotland,0,7130000,Liverpool FC,/fc-liverpool/startseite/verein/31,Leeds United,/leeds-united/startseite/verein/399,out,2000,-7130000
4,Jacob Burns,/jacob-burns/profil/spieler/4096,22,DM,Australia,0,375000,Parramatta Power,/parramatta-power/startseite/verein/2500,Leeds United,/leeds-united/startseite/verein/399,out,2000,-375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,Pedro Porro,/pedro-porro/profil/spieler/553875,23,RB,Spain,40000000,45000000,Sporting CP,/sporting-lissabon/startseite/verein/336,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,out,2023,-5000000
4059,Dejan Kulusevski,/dejan-kulusevski/profil/spieler/431755,23,RW,Sweden,50000000,30000000,Juventus FC,/juventus-turin/startseite/verein/506,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,out,2023,20000000
4060,Arthur Masuaku,/arthur-masuaku/profil/spieler/181380,29,LB,DR Congo,6000000,2000000,West Ham United,/west-ham-united/startseite/verein/379,Besiktas JK,/besiktas-istanbul/startseite/verein/114,in,2023,4000000
4061,Boubacar Traoré,/boubacar-traore/profil/spieler/649020,21,DM,Mali,5000000,11000000,FC Metz,/fc-metz/startseite/verein/347,Wolverhampton Wanderers,/wolverhampton-wanderers/startseite/verein/543,out,2023,-6000000


In [12]:
pl_transfers = pl_transfers[pl_transfers['playerMarketValue'] != 0]
pl_transfers = pl_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
pl_transfers.to_csv('data/pl_transfers.csv', index=False)